In [1]:
import requests
import bs4
import io
import lxml
import pandas as pd
from bs4 import BeautifulSoup 

# World CUP

In [2]:
URL = 'https://stats.espncricinfo.com/ci/engine/records/team/match_results.html?id=14450;type=tournament'
page = requests.get(URL)
bs = BeautifulSoup(page.text, 'lxml')

tables=bs.find_all("table")

summary_df = pd.DataFrame(columns=["Team1","Team2","Winner", "Margin","Ground", "MatchDate"])
rows = tables[0].find_all('tr')
for j,row in enumerate(rows[1::2]):
    cols = row.find_all('td')
    row = []
    for i,col in enumerate(cols):
        if i ==6:
            continue
        else:
            row.append(col.text)
    summary_df.loc[j] = row
    
summary_df.head()

,Team1,Team2,Winner,Margin,Ground,MatchDate
0,Namibia,Sri Lanka,Namibia,55 runs,Geelong,"Oct 16, 2022"
1,Scotland,West Indies,Scotland,42 runs,Hobart,"Oct 17, 2022"
2,Namibia,Netherlands,Netherlands,5 wickets,Geelong,"Oct 18, 2022"
3,Ireland,Scotland,Ireland,6 wickets,Hobart,"Oct 19, 2022"
4,Netherlands,Sri Lanka,Sri Lanka,16 runs,Geelong,"Oct 20, 2022"


In [3]:
tables=bs.find_all("table")
rows = tables[0].find_all('tr')
links = []
for j,row in enumerate(rows[1::2]):
    #print(j,row)
    cols = row.find_all('td')
    for link in cols[6].find_all('a'):
        link = link.get('href')
        link = "https://stats.espncricinfo.com/"+link
        links.append(link)

In [4]:
def batsman_table(table):
    batsmen_df = pd.DataFrame(columns=["Name","Dismissal","Runs", "Balls", "4s", "6s", "SR"])
    rows = table.find_all('tr')
    for j,row in enumerate(rows[1::2]):
        cols = row.find_all('td')
        row = []
        if(len(cols) == 8):
            for i,col in enumerate(cols):
                if(i == 4):
                    continue
                row.append(col.text)
            batsmen_df.loc[j] = row
    return batsmen_df

def bowling_table(table):
    bowler_df = pd.DataFrame(columns=["Name","overs","maidens","Runs","wickets", "econ", "0s","4s","6s", "WD", "NB"])
    rows = table.find_all('tr')
    for j,row in enumerate(rows[1::1]):
        cols = row.find_all('td')
        row = []
        if len(cols) == 11:
            for i,col in enumerate(cols):
                row.append(col.text)
            bowler_df.loc[j] = row
    bowler_df = bowler_df.reset_index(drop=True)
    return bowler_df

def tables(url):
    URL = url
    page = requests.get(URL)
    bs = BeautifulSoup(page.text, 'lxml')

    tables=bs.find_all("table")
    batsmen_df1 = batsman_table(tables[0])
    batsmen_df2 = batsman_table(tables[2])
    bowling_df1 = bowling_table(tables[1])
    bowling_df2 = bowling_table(tables[3])
    
    return batsmen_df1,batsmen_df2,bowling_df1,bowling_df2

In [5]:
tables(links[0])

(                      Name                        Dismissal Runs Balls 4s 6s  \
 0      Michael van Lingen      c Pramod Madushan b Chameera    3     6  0  0   
 1           Divan la Cock       c Shanaka b Pramod Madushan    9     9  1  0   
 2  Jan Nicol Loftie-Eaton           c †Mendis b Karunaratne   20    12  1  2   
 3           Stephan Baard   c DM de Silva b Pramod Madushan   26    24  2  0   
 4      Gerhard Erasmus (c)     c Gunathilaka b PWH de Silva   20    24  0  0   
 5            Jan Frylinck     run out (Gunathilaka/†Mendis)   44    28  4  0   
 6             David Wiese            c †Mendis b Theekshana    0     1  0  0   
 7                 JJ Smit                          not out    31    16  2  2   
 
        SR  
 0   50.00  
 1  100.00  
 2  166.66  
 3  108.33  
 4   83.33  
 5  157.14  
 6    0.00  
 7  193.75  ,
                            Name                    Dismissal Runs Balls 4s 6s  \
 0              Pathum Nissanka             c Smit b Shikongo    9   

# All teams (LPL)

## batting

In [2]:
URL = 'https://stats.espncricinfo.com/ci/engine/records/averages/batting.html?id=14821;type=tournament'
page = requests.get(URL)
bs = BeautifulSoup(page.text, 'lxml')

tables=bs.find_all("table")

headers = []
for i in tables[0].find_all('th'):
    title = i.text
    headers.append(title)
summary_batting = pd.DataFrame(columns = headers)

rows = tables[0].find_all('tr')
for j,row in enumerate(rows[1::2]):
    cols = row.find_all('td')
    row = []
    for i,col in enumerate(cols):
        row.append(col.text)
    summary_batting.loc[j] = row

In [3]:
summary_batting.head()

,Player,Mat,Inns,NO,Runs,HS,Ave,BF,SR,100,50,0,4s,6s
0,TB Abell,3,3,0,45,33,15.00,34,132.35,0,0,1,6,0
1,Afif Hossain,4,3,0,71,54,23.66,54,131.48,0,1,0,7,1
2,FA Allen,9,6,0,66,47,11.00,46,143.47,0,0,2,5,4
3,Anwar Ali,1,1,0,1,1,1.00,2,50.00,0,0,0,0,0
4,SSD Arachchige,1,1,0,53,53,53.00,31,170.96,0,1,0,3,3


## Bowling

In [4]:
URL = 'https://stats.espncricinfo.com/ci/engine/records/averages/bowling.html?id=14821;type=tournament'
page = requests.get(URL)
bs = BeautifulSoup(page.text, 'lxml')

tables=bs.find_all("table")

headers = []
for i in tables[0].find_all('th'):
    title = i.text
    headers.append(title)
summary_bowling = pd.DataFrame(columns = headers)

rows = tables[0].find_all('tr')
for j,row in enumerate(rows[1::2]):
    cols = row.find_all('td')
    row = []
    for i,col in enumerate(cols):
        row.append(col.text)
    summary_bowling.loc[j] = row

In [5]:
summary_bowling = summary_bowling.replace('-', 0)
summary_batting = summary_batting.replace('-', 0)

In [6]:
summary_bowling.head()

,Player,Mat,Inns,Overs,Mdns,Runs,Wkts,BBI,Ave,Econ,SR,4,5,Ct,St
0,TB Abell,3,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Afif Hossain,4,0,0,0,0,0,0,0,0,0,0,0,2,0
2,FA Allen,9,9,24.4,0,156,10,2/1,15.60,6.32,14.8,0,0,3,0
3,Anwar Ali,1,1,4.0,0,44,3,3/44,14.66,11.00,8.0,0,0,2,0
4,SSD Arachchige,1,0,0,0,0,0,0,0,0,0,0,0,0,0


In [7]:
summary_bowling.columns

Index(['Player', 'Mat', 'Inns', 'Overs', 'Mdns', 'Runs', 'Wkts', 'BBI', 'Ave',
       'Econ', 'SR', '4', '5', 'Ct', 'St'],
      dtype='object')

In [8]:
to_float_bat =  ['Ave','SR','Runs']
to_int_bat  = ['Runs','Inns','Inns','NO','Runs','100',
       '50', '0', '4s', '6s']

for column in summary_batting.columns:
    if column in to_float_bat:
        summary_batting[column] = summary_batting[column].astype(float)
    if column in to_int_bat:
        summary_batting[column] = summary_batting[column].astype(int)

        
to_float_bowl =  ['Overs','Ave','Econ','SR']
to_int_bowl  = ['Inns', 'Ct', 'Mdns', 'Runs', 'Wkts',]


for column in summary_bowling.columns:
    if column in to_float_bowl:
        summary_bowling[column] = summary_bowling[column].astype(float)
    if column in to_int_bowl:
        summary_bowling[column] = summary_bowling[column].astype(int)


In [9]:
summary_batting.sort_values(by=['Runs'],ascending=False).head()

,Player,Mat,Inns,NO,Runs,HS,Ave,BF,SR,100,50,0,4s,6s
32,WIA Fernando,10,10,1,339,54,37.66,272,124.63,0,3,0,37,6
75,S Samarawickrama,9,9,4,294,62*,58.80,224,131.25,0,1,0,33,2
14,LD Chandimal,11,11,2,287,63,31.88,226,126.99,0,1,1,26,8
33,ADS Fletcher,9,9,2,266,102*,38.00,202,131.68,1,0,1,29,11
56,PHKD Mendis,10,9,1,260,58,32.50,220,118.18,0,2,0,32,3


In [10]:
summary_bowling.sort_values(by=['Wkts'],ascending=False).head()

,Player,Mat,Inns,Overs,Mdns,Runs,Wkts,BBI,Ave,Econ,SR,4,5,Ct,St
13,CR Brathwaite,8,8,26.0,2,178,18,4/14,9.88,6.84,8.6,2,0,0,0
85,N Thushara,9,9,33.5,1,252,14,3/29,18.00,7.44,14.5,0,0,1,0
72,CAK Rajitha,8,8,26.2,1,166,13,5/22,12.76,6.30,12.1,1,1,3,0
90,V Viyaskanth,8,8,32.0,0,214,13,3/24,16.46,6.68,14.7,0,0,4,0
27,B Fernando,5,5,19.0,2,135,12,4/22,11.25,7.10,9.5,1,0,0,0


In [11]:
summary_batting[(summary_batting['Runs']>80) & (summary_batting['Ave']>25) & (summary_batting['SR']>120)]

,Player,Mat,Inns,NO,Runs,HS,Ave,BF,SR,100,50,0,4s,6s
12,RS Bopara,10,9,4,245,47*,49.00,176,139.20,0,0,0,21,9
14,LD Chandimal,11,11,2,287,63,31.88,226,126.99,0,1,1,26,8
15,JM Cox,8,8,0,205,77,25.62,137,149.63,0,1,1,22,9
30,MNK Fernando,9,8,1,211,63*,30.14,161,131.05,0,2,0,15,9
32,WIA Fernando,10,10,1,339,54,37.66,272,124.63,0,3,0,37,6
33,ADS Fletcher,9,9,2,266,102*,38.00,202,131.68,1,0,1,29,11
44,C Karunaratne,10,6,4,109,32,54.50,67,162.68,0,0,0,7,5
54,AD Mathews,10,9,3,213,73*,35.50,173,123.12,0,1,0,19,7
55,BKG Mendis,9,9,0,256,72,28.44,209,122.48,0,3,1,27,7
70,Rahmanullah Gurbaz,9,9,0,249,73,27.66,158,157.59,0,2,0,26,15


In [12]:
summary_bowling[(summary_bowling['Wkts']>6) & (summary_bowling['Econ']<7.2) & (summary_bowling['SR']<16)]

,Player,Mat,Inns,Overs,Mdns,Runs,Wkts,BBI,Ave,Econ,SR,4,5,Ct,St
2,FA Allen,9,9,24.4,0,156,10,2/1,15.60,6.32,14.8,0,0,3,0
13,CR Brathwaite,8,8,26.0,2,178,18,4/14,9.88,6.84,8.6,2,0,0,0
27,B Fernando,5,5,19.0,2,135,12,4/22,11.25,7.10,9.5,1,0,0,0
34,MW Forde,3,3,9.0,0,50,7,4/11,7.14,5.55,7.7,1,0,1,0
72,CAK Rajitha,8,8,26.2,1,166,13,5/22,12.76,6.30,12.1,1,1,3,0
90,V Viyaskanth,8,8,32.0,0,214,13,3/24,16.46,6.68,14.7,0,0,4,0


In [13]:
overall_df = summary_batting.merge(summary_bowling, left_on='Player', right_on='Player')

In [14]:
overall_df.head()

,Player,Mat_x,Inns_x,NO,Runs_x,HS,Ave_x,BF,SR_x,100,...,Runs_y,Wkts,BBI,Ave_y,Econ,SR_y,4,5,Ct,St
0,TB Abell,3,3,0,45,33,15.00,34,132.35,0,...,0,0,0,0.00,0.00,0.0,0,0,0,0
1,Afif Hossain,4,3,0,71,54,23.66,54,131.48,0,...,0,0,0,0.00,0.00,0.0,0,0,2,0
2,FA Allen,9,6,0,66,47,11.00,46,143.47,0,...,156,10,2/1,15.60,6.32,14.8,0,0,3,0
3,Anwar Ali,1,1,0,1,1,1.00,2,50.00,0,...,44,3,3/44,14.66,11.00,8.0,0,0,2,0
4,SSD Arachchige,1,1,0,53,53,53.00,31,170.96,0,...,0,0,0,0.00,0.00,0.0,0,0,0,0


In [15]:
overall_df.columns

Index(['Player', 'Mat_x', 'Inns_x', 'NO', 'Runs_x', 'HS', 'Ave_x', 'BF',
       'SR_x', '100', '50', '0', '4s', '6s', 'Mat_y', 'Inns_y', 'Overs',
       'Mdns', 'Runs_y', 'Wkts', 'BBI', 'Ave_y', 'Econ', 'SR_y', '4', '5',
       'Ct', 'St'],
      dtype='object')

In [16]:
def points(df):
    battingpoint = df[0] + 16*df[1]+8*df[2]+df[3]+2*df[4]
    srpoint = 0
    if (df[5]>170):
        srpoint = 6
    elif(df[5]<170 and df[5]>150):
        srpoint = 4
    elif(df[5]<150 and df[5]>130):
        srpoint = 3
    elif(df[5]<130 and df[5]>100):
        srpoint = 2
    elif(df[5]<100):
        srpoint = -1
    wktpoints = 0
    if (df[6]>170):
        wktpoints = 6
    elif(df[6]<170 and df[5]>150):
        wktpoints = 4
    
    
    return (battingpoint + df[6]*srpoint+df[7]*25+ df[8]*8)
    

In [17]:
overall_df['points'] = overall_df[['Runs_x','100', '50','4s', '6s','SR_x','Inns_x','Wkts','Mdns']].apply(points,axis=1)

In [18]:
overall_df.head()

,Player,Mat_x,Inns_x,NO,Runs_x,HS,Ave_x,BF,SR_x,100,...,Wkts,BBI,Ave_y,Econ,SR_y,4,5,Ct,St,points
0,TB Abell,3,3,0,45,33,15.00,34,132.35,0,...,0,0,0.00,0.00,0.0,0,0,0,0,60.0
1,Afif Hossain,4,3,0,71,54,23.66,54,131.48,0,...,0,0,0.00,0.00,0.0,0,0,2,0,97.0
2,FA Allen,9,6,0,66,47,11.00,46,143.47,0,...,10,2/1,15.60,6.32,14.8,0,0,3,0,347.0
3,Anwar Ali,1,1,0,1,1,1.00,2,50.00,0,...,3,3/44,14.66,11.00,8.0,0,0,2,0,75.0
4,SSD Arachchige,1,1,0,53,53,53.00,31,170.96,0,...,0,0,0.00,0.00,0.0,0,0,0,0,76.0


In [19]:
min = overall_df['Ave_x'].min()
max = overall_df['Ave_x'].max()
min,max

(0.0, 58.8)

In [20]:
overall_df.sort_values(by=['points'],ascending=False).head(7)

,Player,Mat_x,Inns_x,NO,Runs_x,HS,Ave_x,BF,SR_x,100,...,Wkts,BBI,Ave_y,Econ,SR_y,4,5,Ct,St,points
13,CR Brathwaite,8,7,3,58,20*,14.50,48,120.83,0,...,18,4/14,9.88,6.84,8.6,2,0,0,0,546.0
32,WIA Fernando,10,10,1,339,54,37.66,272,124.63,0,...,0,0,0.00,0.00,0.0,0,0,3,0,432.0
22,PWH de Silva,10,7,2,118,77*,23.60,72,163.88,0,...,10,4/14,21.80,6.41,20.4,1,0,6,0,422.0
56,PHKD Mendis,10,9,1,260,58,32.50,220,118.18,0,...,2,2/26,36.00,8.00,27.0,0,0,3,0,382.0
75,S Samarawickrama,9,9,4,294,62*,58.80,224,131.25,0,...,0,0,0.00,0.00,0.0,0,0,8,2,366.0
33,ADS Fletcher,9,9,2,266,102*,38.00,202,131.68,1,...,0,0,0.00,0.00,0.0,0,0,7,0,360.0
85,N Thushara,9,4,3,5,2*,5.00,8,62.50,0,...,14,3/29,18.00,7.44,14.5,0,0,1,0,359.0


In [101]:
overall_df.loc[2]

Player    FA Allen
Mat_x            9
Inns_x           6
NO               0
Runs_x          66
HS              47
Ave_x         11.0
BF              46
SR_x        143.47
100              0
50               0
0                2
4s               5
6s               4
Mat_y            9
Inns_y           9
Overs         24.4
Mdns             0
Runs_y         156
Wkts            10
BBI            2/1
Ave_y         15.6
Econ          6.32
SR_y          14.8
4                0
5                0
Ct               3
St               0
points       347.0
Name: 2, dtype: object